In [1]:
import numpy as np 
import pandas as pd 
from io import open
import tensorflow as tf
import glob
import pickle
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import time
import copy


In [2]:
!pip install music21
!pip install miditoolkit 
!pip install miditok

     |████████████████████████████████| 19.2 MB 4.1 MB/s eta 0:00:01
  Created wheel for music21: filename=music21-6.7.1-py3-none-any.whl size=21941692 sha256=0b064c94c61f5fe4a69cf53c617b27b3e05431609e4a81953872f4a75d16bff2
  Stored in directory: /root/.cache/pip/wheels/72/44/61/90e4e65262ca1b4d9f707527b540729ce3f64e00fc6b38d54c
Successfully built music21
     |████████████████████████████████| 51 kB 2.9 MB/s eta 0:00:011
  Created wheel for miditoolkit: filename=miditoolkit-0.1.14-py3-none-any.whl size=19537 sha256=1ad5453d7e400a2ecdc3b5efc95ef86b6d31748db0e7331ccf476140b121ac91
  Stored in directory: /root/.cache/pip/wheels/00/ac/58/1527193cdf6d74e3d056086f1be24f4fcb465c45212624b60c
Successfully built miditoolkit
     |████████████████████████████████| 46 kB 3.0 MB/s  eta 0:00:01
     |████████████████████████████████| 15.7 MB 5.8 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled 

In [3]:
from music21 import converter, instrument, note, chord, stream
from miditok import CPWordEncoding, REMIEncoding
from miditoolkit import MidiFile

In [4]:
# Creates the tokenizer and list the file paths
remi_enc = REMIEncoding()  # uses defaults parameters
filename = 'beeth'
files_paths = list(glob.glob('../input/classical-music-midi/'+filename+'/*.mid'))
# files_paths.remove('../input/classical-music-midi/beeth/waldstein_1.mid')
# files_paths.remove('../input/classical-music-midi/beeth/beethoven_les_adieux_2.mid')
# files_paths.remove('../input/classical-music-midi/beeth/beethoven_les_adieux_1.mid')
# files_paths.remove('../input/classical-music-midi/beeth/mond_1.mid')
# files_paths.remove('../input/classical-music-midi/beeth/beethoven_opus10_2.mid')
# files_paths.remove('../input/classical-music-midi/beeth/beethoven_opus10_1.mid')
# files_paths.remove('../input/classical-music-midi/beeth/beethoven_opus10_3.mid')
# files_paths.remove('../input/classical-music-midi/beeth/beethoven_opus22_1.mid')
# files_paths.remove('../input/classical-music-midi/beeth/pathetique_3.mid')

# A validation method to make sure to discard MIDIs we do not want
def midi_valid(midi) -> bool:
    if any(ts.numerator != 4 or ts.denominator !=4 for ts in midi.time_signature_changes):
        return False  # time signature different from 4/4
    if max(note.end for note in midi.instruments[0].notes) < 10 * midi.ticks_per_beat:
        return False  # this MIDI is too short
    return True

# Converts MIDI files to tokens saved as JSON files
# remi_enc.tokenize_midi_dataset(files_paths, './', midi_valid)

In [5]:
notes = []

remi_enc = REMIEncoding()
# glob.glob('../input/classical-music-midi/'+filename+'/*.mid')
for file in files_paths:

    # read the MIDI file
#     midi = converter.parse(file)      
    midi = MidiFile(file)

    # Converts MIDI to tokens
    tokens, track_info = remi_enc.midi_to_tokens(midi)
    
    notes.append(tokens)

In [6]:
# all the beethoven files have 2 tracks (instruments) - Piano left and right
# for now, we will only be using for piano right since it determines the melody
midi.instruments

[Instrument(program=1, is_drum=False, name="Piano right"),
 Instrument(program=1, is_drum=False, name="Piano left")]

In [7]:
piano_right_notes = [note[0] for note in notes]

In [ ]:
piano_right_notes = [np.asarray(note) for note in piano_right_notes]
piano_right_notes = np.array(piano_right_notes, dtype='object')

In [8]:
def preprocess_input(filename, folder=False):
    # master list of notes
    notes = []
    
    # converting folders with multiple MIDI files
    if folder == True:
        assert os.path.exists('../input/classical-music-midi/'+filename)
        for file in glob.glob('../input/classical-music-midi/'+filename+'/*.mid'):
            notes_per_piece = []
            # read the MIDI file
            midi = converter.parse(file)            
            
            try: # file has instrument parts
                s2 = instrument.partitionByInstrument(midi)
                notes_to_parse = s2.parts[1].recurse() 
            except: # file has notes in a flat structure
                notes_to_parse = midi.flat.notes
            
#             print(notes_to_parse)
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes_per_piece.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes_per_piece.append('.'.join(str(n) for n in element.normalOrder))
            notes.append(notes_per_piece)
    else:
        assert os.path.exists(filename)
        midi = converter.parse(filename)
        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[1].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
#         print(notes_to_parse)
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
        

    with open('./notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
#     print(notes)
    return notes

In [ ]:
notes = preprocess_input('debussy', folder=True)

In [ ]:
notes = []
with (open("../input/beethoven-notes/notes", "rb")) as openfile:
    while True:
        try:
            notes = (pickle.load(openfile))
        except EOFError:
            break
# notes = 

In [ ]:
notes

In [9]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)
    
    def words(self):
        return self.idx2word


class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self.tokenize(piano_right_notes[:-12])
        self.valid = self.tokenize(piano_right_notes[-12:-6])
        self.test = self.tokenize(piano_right_notes[-6:])
     
    def tokenize(self, notes):
        """Tokenizes a note sequence"""
        assert len(notes) > 0
        
#         Add notes to the dictionary
        for song in notes:
            for note in song:
                self.dictionary.add_word(note)
#         # Add words to the dictionary
#         with open(path, 'r', encoding="utf8") as f:
#             for line in f:
#                 words = line.split() + ['<eos>']
#                 for word in words:
#                     self.dictionary.add_word(word)

        # Tokenize file content
        idss = []
        
        for song in notes:
            ids = []
            for note in song:
#                 print(note)
                ids.append(self.dictionary.word2idx[note])
        idss.append(torch.tensor(ids).type(torch.int64))
        ids = torch.cat(idss)
            
#         with open(path, 'r', encoding="utf8") as f:
#             idss = []
#             for line in f:
#                 words = line.split() + ['<eos>']
#                 ids = []
#                 for word in words:
#                     ids.append(self.dictionary.word2idx[word])
#                 idss.append(torch.tensor(ids).type(torch.int64))
#             ids = torch.cat(idss)

        return ids

In [10]:
corpus = Corpus('../input/classical-music-midi/beeth')

In [11]:
corpus.train.size(0)

7077

In [12]:
device = 'cpu'
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

eval_batch_size = 10
batch_size = 40
train_data = batchify(corpus.train, batch_size)
val_data = batchify(corpus.valid, eval_batch_size)
test_data = batchify(corpus.test, batch_size)


In [13]:
test_data.size()

torch.Size([132, 40])

In [14]:
class PositionalEncoding(nn.Module):
    """Inject some information about the relative or absolute position of the tokens
        in the sequence. The positional encodings have the same dimension as
        the embeddings, so that the two can be summed. Here, we use sine and cosine
        functions of different frequencies.
    .. math::
        \text{PosEncoder}(pos, 2i) = sin(pos/10000^(2i/d_model))
        \text{PosEncoder}(pos, 2i+1) = cos(pos/10000^(2i/d_model))
        \text{where pos is the word position and i is the embed idx)
    Args:
        d_model: the embed dim (required).
        dropout: the dropout value (default=0.1).
        max_len: the max. length of the incoming sequence (default=5000).
    Examples:
        >>> pos_encoder = PositionalEncoding(d_model)
    """

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.d_model = d_model

        # PE is the Positional Encoding matrix 
        # THIS STORES THE POSITIONS OF THE SEQUENCE
        pe = torch.zeros(max_len, d_model)
        
        # Arange - RETURNS A RANGE BETWEEN VALUES, HERE IT IS 0 - max_len
        # unsqueeze - adds a dimension, 1 means that each element in the first list is now in a list
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        
        # division term, here it is (10000 ** ((2 * i)/d_model))
        
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        # calculating the position encoding for the even and odd terms
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # Unsqueeze 0 will put PE in one list
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """Inputs of forward function
        Args:
            x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        Examples:
            >>> output = pos_encoder(x)
        """
        # make embeddings relatively larger
        # This is so we do not lose the importance of the embedding
        x = x * math.sqrt(self.d_model)
        # we add the embedding to the PE 
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [15]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

        self.d_model = d_model
    def forward(self, x):
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        # make embeddings relatively larger
        # This is so we do not lose the importance of the embedding
        x = x * math.sqrt(self.d_model)
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [16]:
class MultiHeadAttention(nn.Module):
    """Parallel self attention given the number of heads and size
    .. math::
        X * Wk = K
        X * Wq = Q
        X * Wv = V
        \text{attention} = softmax(Q * transpose(K) / sqrt(d_model)) * V
    Args:
        d_model: the embed dim (default = 256).
        heads: number of heads (default = 4)
        max_length = max length of sequences (default = 2048)
        dropout: the dropout value (default = 0.1).
    Examples:
        >>> attention = MultiHeadAttention(d_model, heads)
    """
    def __init__(self, d_model = 256, heads = 4, max_length = 2048, dropout = 0.1):
        super().__init__()
        
        self.d = d_model
        self.h = heads
        self.dh = d_model // heads
        self.max_length = max_length
        self.E = torch.randn([heads, self.max_length, self.dh])
        
        self.q_linear = nn.Linear(self.dh, self.dh)
        self.v_linear = nn.Linear(self.dh, self.dh)
        self.k_linear = nn.Linear(self.dh, self.dh)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
    
    def forward(self, q, k, v, mask=None):
        
        batch_size = q.size(0)
        T = q.size(1)
        dh = q.size(2) // self.h
        
        k = k.view(batch_size, T, self.h, dh)
        q = q.view(batch_size, T, self.h, dh)
        v = v.view(batch_size, T, self.h, dh)
        # perform linear operation and split into h heads
        
        K = self.k_linear(k)
        Q = self.q_linear(q)
        V = self.v_linear(v)
        
        # transpose to get dimensions bs * h * sl * d_model
       
        K = K.transpose(1,2)
        Q = Q.transpose(1,2)
        V = V.transpose(1,2)

        #start index of position embedding
        
        embedding_start = self.max_length - T
        
        #apply same position embeddings across the batch
        
        Er  = self.E[:, embedding_start:, :].unsqueeze(0)
        
        QE = torch.matmul(Q, K.transpose(-2, -1))
        QE = self.mask_attention_positions(QE)
        
        #Get relative position attention scores
        #combine batch with head dimension
        SRel = self.skew_padding_position(QE)

#         Q = Q.contiguous().view(batch_size*self.h, T, self.dh)
#         K = K.contiguous().view(batch_size*self.h, T, self.dh)
#         V = V.contiguous().view(batch_size*self.h, T, self.dh)
        
        #Compute scaled dot-product self-attention
        #scale pre-matrix multiplication   
        Q = Q / (q.size(2) ** (1/4))
        K = K / (q.size(2) ** (1/4))

        # calculate attention 
        scores, weights = self.attention(QE, SRel, V, mask, self.dropout)
        
        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).contiguous().view(batch_size, -1, self.d)
        
        output = self.out(concat)
    
        return output

    def attention(self, QE, Srel, V, mask=None, dropout=None):
        log = QE + Srel
        log = log / math.sqrt(self.dh)

        if mask is not None:
#             mask = mask.unsqueeze(1)
            log += (mask.to(torch.int64) * -1e9).to(log.dtype)

#             print(mask.shape)
#             log = log.masked_fill(mask == 0, float('-inf'))

        scores = F.softmax(log, -1)
        
        if dropout is not None:
            scores = dropout(scores)
            
#         print(scores.shape, V.shape)
        attention = torch.matmul(scores, V)
        
#         scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)

#         if mask is not None:
#             print(scores.shape)

#     #         mask = mask.unsqueeze(1)
#             print(mask.shape)
#     #         scores = scores.masked_fill(mask == 0, float('-inf'))
#         scores = F.softmax(scores, dim=-1)

#         

#         output = torch.matmul(scores, v)
        return attention, scores
    
    def mask_attention_positions(self, qe):
        # to avoid looking backward by masking the positions
        index = qe.shape[-1]
        mask = torch.triu(torch.ones(index, index), 1).flip(1)
        return qe.masked_fill((mask == 1), 0)
    
    def skew_padding_position(self, qe):
        # to add padding to the skewed result after masking the matrix
        # column of zeros on left
        padded_qe = F.pad(qe, [1,0])
        s = padded_qe.shape
        padded_qe = padded_qe.view(s[0], s[1], s[3], s[2])
        #take out first (padded) row
        return padded_qe[:,:,1:,:]

In [17]:
class FeedForward(nn.Module):
#     def __init__(self, d_model, d_ff=2048, dropout = 0.1):
#         super().__init__() 
#         # We set d_ff as a default to 2048
#         self.linear_1 = nn.Linear(d_model, d_ff)
#         self.dropout = nn.Dropout(dropout)
#         self.linear_2 = nn.Linear(d_ff, d_model)
#     def forward(self, x):
#         x = self.dropout(F.relu(self.linear_1(x)))
#         x = self.linear_2(x)
#         return x
    def __init__(self, d_model, d_ff=2048, dropout=0.1):
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.linear_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.linear_2(self.dropout(F.relu(self.linear_1(x))))


In [18]:
# build an encoder layer with one multi-head attention layer and one # feed-forward layer
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout = 0.1):
        super().__init__()
        self.norm_1 = nn.LayerNorm(d_model)
        self.norm_2 = nn.LayerNorm(d_model)
        
        self.attn = MultiHeadAttention(d_model, heads)
        self.ff = FeedForward(d_model)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn(x2,x2,x2,mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))
        return x
    
# build a decoder layer with two multi-head attention layers and
# one feed-forward layer
class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = nn.LayerNorm(d_model)
        self.norm_2 = nn.LayerNorm(d_model)
        self.norm_3 = nn.LayerNorm(d_model)
        
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        self.dropout_3 = nn.Dropout(dropout)
        
        self.attn_1 = MultiHeadAttention(d_model, heads)
        self.attn_2 = MultiHeadAttention(d_model, heads)
        self.ff = FeedForward(d_model)
    def forward(self, x, memory, mask = None):
#         x2 = self.norm_1(x)
#         x = x + self.dropout_1(self.attn_1(x2, x2, x2, trg_mask))
#         x2 = self.norm_2(x)
#         x = x + self.dropout_2(self.attn_2(x2, x2, x2,
#         src_mask))
#         x2 = self.norm_3(x)
#         x = x + self.dropout_3(self.ff(x2))
        
        #perform masked attention on input
        #masked so queries cannot attend to subsequent keys
        #Pass through sublayers of attention and feedforward.
        #Apply dropout to sublayer output, add it to input, and norm.
        attn = self.attn_1(x, x, x, mask)
        x = x + self.dropout_1(attn)
        x = self.norm_1(x)
        x = x + self.dropout_2(self.attn_2(x, memory, memory, mask))
        ff = self.ff(x)
        x = x + self.dropout_2(ff)
        x = self.norm_2(x)

        return x
#         return x
# We can then build a convenient cloning function that can generate multiple layers:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [19]:
class Encoder(nn.Module):
    def __init__(self, d_model, N, heads):
        super().__init__()
        self.N = N
        self.layers = get_clones(EncoderLayer(d_model, heads), self.N)
        self.norm = nn.LayerNorm(d_model)
    def forward(self, src, mask):
        x = self.layers[0](src, mask)
        for i in range(1,self.N):
            x = self.layers[i](x, mask)
        return self.norm(x)
    
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N
        self.layers = get_clones(DecoderLayer(d_model, heads), N)
        self.norm = nn.LayerNorm(d_model)
    def forward(self, src, trg, src_mask = None):
        x = self.layers[0](src, trg, src_mask)
        for i in range(1,self.N):
            x = self.layers[i](x, trg, src_mask)
        return self.norm(x)

In [67]:
class TransformerModel(nn.Module):
    """Container module with an encoder, a recurrent or transformer module, and a decoder."""

    def __init__(self, ntoken, d_model, nhead, nhid, nlayers, dropout=0.5, max_length = 2048):
        super(TransformerModel, self).__init__()
        try:
            from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        except:
            raise ImportError('TransformerEncoder module does not exist in PyTorch 1.1 or lower.')
        self.model_type = 'Transformer'
        # original mask
        self.src_mask = None
        self.max_length = max_length
        self.d_model = d_model
        
        # embedding encoding
        self.embedding = nn.Embedding(ntoken, d_model)
        
        # positional encoding
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        
        # encoder
        encoder_layers = TransformerEncoderLayer(d_model, nhead, nhid, dropout)
        self.encoder = TransformerEncoder(encoder_layers, nlayers)
        
#         self.decoder = nn.Linear(d_model, ntoken)


        
#         self.encoder = Encoder(d_model, nlayers, nhead)
#         self.encoder.eval()
        # decoder
        decoder_layers = TransformerDecoderLayer(d_model, nhead, nhid, dropout)
        self.decoder = TransformerDecoder(decoder_layers, nlayers)
#         self.decoder = Decoder(ntoken, d_model, nlayers, nhead)
#         self.decoder.eval()
        # classification layer
        self.classification_layer = nn.Linear(d_model, ntoken)
        
        self.init_weights()
    def _generate_square_subsequent_mask(self, sz):
        
        mask = torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)
#         mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
#         mask = utils.get_masked_with_pad_tensor(self.max_length, x, x, config.pad_token)
        return mask

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.embedding.weight, -initrange, initrange)
        self.classification_layer.bias.data.zero_()
        self.classification_layer.weight.data.uniform_(-initrange, initrange)
#         nn.init.zeros_(self.decoder.weight)
#         nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, src, has_mask=True):
        if has_mask:
            device = src.device
            if self.src_mask is None or self.src_mask.size(0) != len(src):
#                 mask = self._generate_square_subsequent_mask(src.shape[1]).to(device)
#                 mask = utils.get_masked_with_pad_tensor(self.max_length, src, src, self.d_model)
                mask = self._generate_square_subsequent_mask(len(src)).to(device)
                self.src_mask = mask
        else:
            self.src_mask = None

        src = self.pos_encoder(self.embedding(src))
        output = self.encoder(src, self.src_mask)
        output = self.decoder(output, src)
        #Flatten:
        shape = output.shape
        tensor_reshaped = output.reshape(shape[0],-1)
        #Drop all rows containing any nan:
        tensor_reshaped = tensor_reshaped[~torch.any(tensor_reshaped.isnan(),dim=1)]
        #Reshape back:
        output = tensor_reshaped.reshape(tensor_reshaped.shape[0],*shape[1:])
        
        output = self.classification_layer(output)
        return F.log_softmax(output, dim=-1)


In [68]:
ntokens = len(corpus.dictionary)
emsize = 100
nhead = 2
nhid = 100
nlayer = 2
dropout = 0.2
# Loop over epochs.
lr = 5
best_val_loss = None
epochs = 400
save = './model.pt'
criterion = nn.CrossEntropyLoss()

In [51]:
final_losses = []
emsizes = []
nheads = []
nhids = []
nlayers = []

for em in emsize:
    for head in nhead:
        for hid in nhid:
            for layer in nlayer:
                emsizes.append(em)
                nheads.append(head)
                nhids.append(hid)
                nlayers.append(layer)
                best_val_loss = None

                model = TransformerModel(ntokens, em, head, hid, layer, dropout).to(device)

                # At any point you can hit Ctrl + C to break out of training early.
                try:
                    for epoch in range(1, epochs+1):
                        epoch_start_time = time.time()
                        train()
                        val_loss = evaluate(val_data)
                        print('-' * 89)
                        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
                                'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                                           val_loss, math.exp(val_loss)))
                        print('-' * 89)
                        # Save the model if the validation loss is the best we've seen so far.
                        if not best_val_loss or val_loss < best_val_loss:
                            with open(save, 'wb') as f:
                                torch.save(model, f)
                            best_val_loss = val_loss
                        else:
                            # Anneal the learning rate if no improvement has been seen in the validation dataset.
                            lr /= 2.0
                except KeyboardInterrupt:
                    print('-' * 89)
                    print('Exiting from training early')
                final_losses.append(best_val_loss)

    
scores = list(zip(emsizes, nheads, nhids, nlayers, final_losses))
losses_df = pd.DataFrame(scores, columns=['embedding_size', 'num_heads', 'num_hidden', 'num_layers', 'best_loss'])

TypeError: 'int' object is not iterable

In [52]:
scores = list(zip(emsizes, nheads, nhids, nlayers, final_losses))
losses_df = pd.DataFrame(scores, columns=['embedding_size', 'num_heads', 'num_hidden', 'num_layers', 'best_loss'])

In [69]:
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayer, dropout).to(device)

In [70]:
def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""

    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

In [71]:
# get_batch subdivides the source data into chunks of length args.bptt.
# If source is equal to the example output of the batchify function, with
# a bptt-limit of 2, we'd get the following two Variables for i = 0:
# ┌ a g m s ┐ ┌ b h n t ┐
# └ b h n t ┘ └ c i o u ┘
# Note that despite the name of the function, the subdivison of data is not
# done along the batch dimension (i.e. dimension 1), since that was handled
# by the batchify function. The chunks are along dimension 0, corresponding
# to the seq_len dimension in the LSTM.
seq_length = 40
def get_batch(source, i):
    seq_len = min(seq_length, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

In [72]:
def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0.
    ntokens = len(corpus.dictionary)

    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, seq_length):
            data, targets = get_batch(data_source, i)
            
            output = model(data)
            output = output.view(-1, ntokens)
            
            total_loss += len(data) * criterion(output, targets).item()
    return total_loss / (len(data_source) - 1)

In [73]:
def train():
    # Turn on training mode which enables dropout.
    model.train()
    total_loss = 0.
    start_time = time.time()
    ntokens = len(corpus.dictionary)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    log_interval = 100
    
    for batch, i in enumerate(range(0, train_data.size(0) - 1, seq_length)):
        
        data, targets = get_batch(train_data, i)
        batch_size = data.size(0)

        output = model(data)
        loss = criterion(output.view(-1, ntokens), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        
        
        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

In [75]:
# At any point you can hit Ctrl + C to break out of training early.
try:
    for epoch in range(1, epochs+1):
        epoch_start_time = time.time()
        train()
        val_loss = evaluate(val_data)
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
                'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                           val_loss, math.exp(val_loss)))
        print('-' * 89)
        # Save the model if the validation loss is the best we've seen so far.
        if not best_val_loss or val_loss < best_val_loss:
            with open(save, 'wb') as f:
                torch.save(model, f)
            best_val_loss = val_loss
        else:
            # Anneal the learning rate if no improvement has been seen in the validation dataset.
            lr /= 2.0
except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')
    

-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  0.50s | valid loss  4.79 | valid ppl   120.41
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   2 | time:  0.50s | valid loss  4.79 | valid ppl   120.41
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   3 | time:  0.51s | valid loss  4.79 | valid ppl   120.41
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   4 | time:  0.50s | valid loss  4.79 | valid ppl   120.41
--------------------------------------------------------------------------

In [ ]:
n_generate = 500
temperature = 1.0
sequence = []
log_interval = 50 # interval between logs

In [ ]:
model.eval()

input = torch.randint(ntokens, (1, 1), dtype=torch.long).to(device)

with open('./output', 'w') as outf:
    with torch.no_grad():  # no tracking history
        for i in range(n_generate):
            
            output = model(input, False)
            word_weights = output[-1].squeeze().div(temperature).exp().cpu()
            word_idx = torch.multinomial(word_weights, 1)[0]
            word_tensor = torch.Tensor([[word_idx]]).long().to(device)
            input = torch.cat([input, word_tensor], 0)


            word = corpus.dictionary.idx2word[word_idx]

            outf.write(word + ('\n' if i % 20 == 19 else ' '))
            
            sequence.append(word)

            if i % log_interval == 0:
                print('| Generated {}/{} notes'.format(i, n_generate))

In [ ]:
# Creating a MIDI file from prediction
def create_MIDI(gen, name=""):
    
    # the offset is the time difference between notes, we assume its 0.5 here
    offset = 0
    music = []
    
    for seq in gen:
        # chords are seperated by .
        if ('.' in seq) or seq.isdigit():
            chordnotes = seq.split('.')
            n = []
            
            for cur_note in chordnotes:
                new_n = note.Note(int(cur_note))
                new_n.storedInstrument = instrument.Piano() # single piano instrument only
                n.append(new_n)
            
            new_c = chord.Chord(n)
            new_c.offset = offset
            music.append(new_c)
        
        else:            
            new_n = note.Note(seq)
            new_n.storedInstrument = instrument.Piano() # single piano instrument only
            new_n.offset = offset
            music.append(new_n)
        
        offset += 0.5
    
    print(notes)
    # producing a MIDI stream
    midi = stream.Stream(music)
    
    midi.write('midi', fp='new_music_'+name+'.mid')

In [ ]:
create_MIDI(sequence, name='beeth_all_4.31')